In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td><a href="https://colab.research.google.com/github/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/quickstart/google_adk/adk_session_with_cloudsql.ipynb"><img src="https://avatars.githubusercontent.com/u/33467679?s=200&v=4" width="32px" alt="Colab logo"> Run in Colab</a></td>
  <td><a href="https://github.com/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/quickstart/google_adk/adk_session_with_cloudsql.ipynb"><img src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px" alt="GitHub logo"> View on GitHub</a></td>
  <td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/ai-ml-recipes/main/notebooks/quickstart/google_adk/adk_session_with_cloudsql.ipynb"><img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> Open in Vertex AI Workbench</a></td>
  <td><a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/GoogleCloudPlatform/ai-ml-recipes/blob/main/notebooks/quickstart/google_adk/adk_session_with_cloudsql.ipynb"><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35"> Open in BQ Studio</a></td>
  <td><a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fai-ml-recipes%2Fmain%2Fnotebooks/quickstart/google_adk/adk_session_with_cloudsql.ipynb"><img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"> Open in Colab Enterprise</a></td>
</table>

# ADK Session with Cloud SQL

This notebook demonstrates how to build and deploy an Agent Development Kit (ADK) agent that leverages different session services for conversation memory. Specifically, it covers:

1.  Using a local SQLite database for quick development and testing.
2.  Migrating to a production-ready Cloud SQL for PostgreSQL database for robust, scalable session management.

It walks through setting up the agent, configuring both session types, and observing how conversation history is stored.

## Setup

### Install Google Cloud Libraries

This cell installs all the required Python libraries for this notebook:

*   `google-cloud-aiplatform`: The client library for Vertex AI and Agent Engine.
*   `google-adk`: The Agent Development Kit for building the agent.
*   `pandas`: Used for displaying database query results in a clean table format.
*   `pg8000`: A Python driver needed to connect to the PostgreSQL database.

In [ ]:
%pip install -q google-cloud-aiplatform[reasoning_engines] google-adk pandas pg8000

print("✅ All required libraries are installed.")

### Import Libraries

This cell imports the necessary Python libraries and modules required to run the notebook, including components from the Google ADK and Vertex AI SDK.

In [ ]:
import os
import vertexai
import sqlite3
import pandas as pd
from google.adk.agents import Agent
from google.adk.sessions import BaseSessionService, DatabaseSessionService
from vertexai.preview.reasoning_engines import AdkApp
from vertexai import agent_engines

### Configure Google Cloud Environment

This cell sets the core configuration for your Google Cloud environment. You need to replace the placeholder values below with your actual Project ID, Location, and the name of a Cloud Storage bucket for staging.

In [ ]:
os.environ["GOOGLE_CLOUD_PROJECT"] = "your-project-id" # ACTION REQUIRED: Replace with your Project ID
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1" # ACTION REQUIRED: Replace with your preferred region
os.environ["STAGING_BUCKET"] = "gs://your-staging-bucket" # ACTION REQUIRED: Replace with a GCS bucket for staging

print("✅ Google Cloud environment variables configured.")

This cell loads the Google Cloud project ID, location, and staging bucket from the environment variables set in the previous cell into Python variables for use throughout the notebook.

In [ ]:
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]
LOCATION = os.environ["GOOGLE_CLOUD_LOCATION"]
STAGING_BUCKET = os.environ["STAGING_BUCKET"]

This cell initializes the Vertex AI SDK. It authenticates your environment and sets the default project, location, and staging bucket for all subsequent Vertex AI calls made within this session.

In [ ]:
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

## Define the Agent

This section defines the core Agent, which is the brain of our application. It combines a specific Gemini model with a detailed instruction prompt that defines its personality, goals, and constraints.

In [ ]:
simple_prompt = """You are a helpful example bot that tries to answer anything the client wants.
Help the client by answering his questions. Be respectful and kind, but if the client asks something bad feel free to deny it"""

root_agent = Agent(
    name="rag_agent",
    model="gemini-2.0-flash",
    description=(
        "Help the user by answering questions."
    ),
    instruction=simple_prompt,
)

## Option 1: Using SQLite (Local Development & Testing)

To get started, we'll use the simplest method for storing conversation history: a local **SQLite** database.

### Why Use SQLite?
*   **Zero Setup**: It doesn't require any separate database server, authentication, or network configuration. The ADK handles everything for you.
*   **File-Based**: Your entire database is a single file (`sessions.db`) that gets created in your project directory, making it easy to inspect or delete.
*   **Perfect for Development**: It's the ideal choice for quickly building and testing your agent on your local machine.

### Define the Session Service Builder for SQLite

To enable the agent to remember conversations, it requires a session service. This function defines a builder that configures the agent to store conversation history in a local SQLite database file (`sessions.db`). This is ideal for quick local development and testing.

In [ ]:
def create_database_session_service_sqlite() -> BaseSessionService:
    """
    Creates and configures the DatabaseSessionService for SQLite.
    This function acts as a "builder" that the AdkApp can call whenever
    it needs to create a session service instance.
    """
    db_connection_string = "sqlite:///sessions.db"

    print(f"Session service configured to use URL: {db_connection_string}")
    return DatabaseSessionService(db_url=db_connection_string)

### Create the AdkApp Instance with SQLite Session

The AdkApp acts as the main entry point for interacting with our agent. We initialize it by combining our defined agent logic with the SQLite session management service.

In [ ]:
app = AdkApp(
    agent=root_agent,
    session_service_builder=create_database_session_service_sqlite,
)

### Create a New Conversation Session

Before we can send messages, a conversation session must be created. This step generates a unique session ID for a specified user, which will be used to track the conversation history.

In [ ]:
user_id = "local_user_demo"

session = app.create_session(user_id=user_id)

print(f"Created a new session for user '{user_id}' with session ID: {session.id}")

### Send a Message and Stream the Response

This cell sends a user message to the agent within the created session and streams the agent's response. The conversation turn will be saved in the local SQLite database.

In [ ]:
print("\n--- User's Message ---")
message = "Hello whats up?"
print(f"You: {message}\n")

print("--- Agent's Response ---")
for event in app.stream_query(user_id = user_id, 
                              session_id=session.id, 
                              message=message):
    print(event['content'])

### Inspect the SQLite Database

After the conversation, the DatabaseSessionService has persisted the history into the 'sessions.db' file. This section explores the tables created by the ADK to store session data.

In [ ]:
# We have the following tables: app_states, events, sessions, user_states
# Lets take a look inside each one of them

#### Establish Database Connection

This cell establishes a connection to the local 'sessions.db' SQLite file using the `sqlite3` library, allowing us to query and inspect its contents.

In [ ]:
DB_FILE = "sessions.db"

pd.set_option('display.max_colwidth', None)

conn = sqlite3.connect(DB_FILE)

#### Query the 'sessions' Table

The 'sessions' table stores the primary record for each conversation, including the full chat history between the user and the agent.

In [ ]:
print("--- Contents of the 'sessions' table ---")
sessions_df = pd.read_sql_query("SELECT * FROM sessions", conn)
display(sessions_df)

#### Query the 'events' Table

The 'events' table is used by the ADK to log specific actions or occurrences during agent request processing.

In [ ]:
print("--- Contents of the 'events' table ---")
events_df = pd.read_sql_query("SELECT * FROM events", conn)
display(events_df)

#### Query the 'app_states' Table

The 'app_states' table stores the agent's internal application state or tool memory that needs to persist across turns, separate from the chat history.

In [ ]:
print("--- Contents of the 'app_states' table ---")
app_states_df = pd.read_sql_query("SELECT * FROM app_states", conn)
display(app_states_df)

#### Query the 'user_states' Table

The 'user_states' table is designed to hold long-term memory about a specific user, such as preferences, which can be recalled across multiple conversations or sessions.

In [ ]:
print("--- Contents of the 'user_states' table ---")
user_states_df = pd.read_sql_query("SELECT * FROM user_states", conn)
display(user_states_df)

## Option 2: Using a Cloud SQL Database (Production Setup)

While SQLite is great for getting started, a managed cloud database like **Cloud SQL** is the right choice for a more robust or production-ready application.

### Why Use Cloud SQL?
*   **Scalability**: It can handle many more simultaneous connections than a local SQLite file.
*   **Persistence & Reliability**: As a managed service, Google handles backups, failovers, and maintenance, making your data much safer.
*   **Accessibility**: Your deployed applications (e.g., on Agent Engine) can connect to it from anywhere within your Google Cloud project.

The following steps will guide you through creating a PostgreSQL instance on Cloud SQL using `gcloud` commands.

### Step 1: Enable APIs

Before creating an instance, you need to ensure the necessary APIs are enabled for your project. The following command enables the Cloud SQL Admin API (for managing instances) and the Vertex AI API.

This command enables the necessary Google Cloud APIs: the Cloud SQL Admin API for managing database instances and the Vertex AI API for agent deployment and interaction.

In [ ]:
!gcloud services enable sqladmin.googleapis.com aiplatform.googleapis.com --project=$PROJECT_ID

print("✅ APIs enabled.")

### Step 2: Create the Cloud SQL Instance

Next, we'll create the Cloud SQL instance. This command will provision a new PostgreSQL database.

*   **Instance ID**: A unique name for your instance, like `adk-agent-db`.
*   **Password**: You will be prompted to set a strong password for the default `postgres` user. **Important: Save this password somewhere secure!** You will need it for your `.env` file.
*   **Region**: To get the best performance, choose the **same region** where your other Vertex AI resources are located (e.g., `us-central1`).
*   **Database Version**: We'll use PostgreSQL 13, but you can choose another supported version.
*   **Tier**: The `db-g1-small` is a cost-effective option for development and testing.

Replace `adk-agent-db` with your desired instance ID in the command below.

This cell provisions a new PostgreSQL database instance on Cloud SQL. You will be prompted to set a strong password for the 'postgres' user. Ensure the instance ID is unique and the region matches your other Vertex AI resources. This command can take several minutes to complete.

In [ ]:
import getpass
import shlex
import os

INSTANCE_ID = "my-instance-name" # ACTION REQUIRED: Replace with your desired instance ID
REGION = LOCATION 

print(f"Creating Cloud SQL instance '{INSTANCE_ID}'. This will take several minutes...")
!gcloud sql instances create {INSTANCE_ID} \
    --database-version=POSTGRES_13 \
    --tier=db-g1-small \
    --region={REGION} \
    --project={PROJECT_ID}
print("Instance created.")

db_password = getpass.getpass("Enter a secure password for the 'postgres' user: ")

os.environ["DB_PASSWORD"] = db_password
print("DB_PASSWORD environment variable has been set in this session.")

quoted_password = shlex.quote(db_password)

!gcloud sql users set-password postgres \
    --instance={INSTANCE_ID} \
    --project={PROJECT_ID} \
    --password={quoted_password}

print(f"✅ Cloud SQL instance '{INSTANCE_ID}' created and password set for 'postgres' user.")

### Step 3: Create a Database

Once your instance is running, you need to create a specific database inside it for your agent's sessions. We'll name it `agent_sessions`.

This command creates a dedicated database within your Cloud SQL instance to store the agent's session data.

In [ ]:
DATABASE_NAME = "my-database-name" # ACTION REQUIRED: Replace with your desired database name

!gcloud sql databases create {DATABASE_NAME} \
    --instance={INSTANCE_ID} \
    --project={PROJECT_ID}

print(f"✅ Database '{DATABASE_NAME}' created in instance '{INSTANCE_ID}'.")

### Step 4: Get the Instance Connection Name

The instance connection name is a unique identifier for your database instance that is required for connecting securely, especially with the Cloud SQL Auth Proxy. The following command retrieves this name.

This cell retrieves the unique instance connection name, which is essential for securely connecting to your Cloud SQL database via the Cloud SQL Auth Proxy.

In [ ]:
CONNECTION_NAME_VAR = !gcloud sql instances describe {INSTANCE_ID} --project={PROJECT_ID} --format='value(connectionName)'
CONNECTION_NAME = CONNECTION_NAME_VAR[0]

print(f"Instance Connection Name: {CONNECTION_NAME}")
print("ACTION: Copy this value and paste it into the 'Configure Cloud SQL Database Connection Details' cell below.")

### Step 5: Download and Run the Cloud SQL Auth Proxy

To connect your local machine to your Cloud SQL instance securely, you must use the **Cloud SQL Auth Proxy**. This tool creates a secure local connection without needing to configure IP allowlisting.

The next cell automates the process of downloading the correct proxy for your operating system and starting it as a background process. The proxy must be running whenever you want to connect to your database from this notebook.

**Note**: The proxy will run in the background. To stop it, you will need to interrupt or restart the notebook's kernel.

This cell downloads the Cloud SQL Auth Proxy for your operating system and starts it as a background process. The proxy establishes a secure, authenticated connection to your Cloud SQL instance without needing IP whitelisting.

In [ ]:
import os
import platform
import subprocess
import time

if 'CONNECTION_NAME' not in locals():
    raise NameError("CONNECTION_NAME is not defined. Please run the previous cell to get the instance connection name.")

system = platform.system()
machine = platform.machine()
proxy_url = ""
executable_name = "cloud-sql-proxy"

if system == "Linux" and machine == "x86_64":
    proxy_url = "https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.8.2/cloud-sql-proxy.linux.amd64"
elif system == "Darwin":
    if machine == "arm64":
        proxy_url = "https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.8.2/cloud-sql-proxy.darwin.arm64"
    elif machine == "x86_64":
        proxy_url = "https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.8.2/cloud-sql-proxy.darwin.amd64"
elif system == "Windows":
     proxy_url = "https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.8.2/cloud-sql-proxy.x64.exe"
     executable_name += ".exe"

if not proxy_url:
    print(f"Unsupported OS/architecture: {system}/{machine}. Please install the Cloud SQL Auth Proxy manually.")
else:
    print(f"Downloading Cloud SQL Auth Proxy for {system}/{machine}...")
    !curl -o {executable_name} {proxy_url}
    
    if system != "Windows":
        print("Making it executable...")
        !chmod +x {executable_name}

    print(f"Starting proxy for instance: {CONNECTION_NAME}")
    proxy_process = subprocess.Popen([f"./{executable_name}", CONNECTION_NAME])
    
    time.sleep(5)
    
    print(f"✅ Cloud SQL Auth Proxy started in the background with PID: {proxy_process.pid}")
    print("You can now proceed to the next cells to connect to the database.")

### Configure Cloud SQL Database Connection Details

This cell sets up the necessary environment variables and Python variables for connecting to your Cloud SQL database. Ensure you replace the placeholder values for `DB_NAME`, `CONNECTION_NAME`, `DB_HOST`, and `DB_PORT` with your actual Cloud SQL details. The `DB_PASSWORD` should have been set interactively earlier.

In [ ]:
os.environ["DB_USER"] = "postgres"
os.environ["DB_NAME"] = "my-database-name" # ACTION REQUIRED: Use the database name from Step 3
os.environ["CONNECTION_NAME"] = "your-project-id:your-region:my-instance-name" # ACTION REQUIRED: Paste Connection Name from Step 4
os.environ["DB_HOST"] = "127.0.0.1" # Use localhost since Cloud SQL Auth Proxy is running locally
os.environ["DB_PORT"] = "5432" # Default PostgreSQL port

DB_USER = os.environ["DB_USER"]
DB_PASSWORD = os.environ["DB_PASSWORD"]
DB_NAME = os.environ["DB_NAME"]
CONNECTION_NAME = os.environ["CONNECTION_NAME"]
DB_HOST = os.environ["DB_HOST"]
DB_PORT = os.environ["DB_PORT"]

if not DB_PASSWORD:
    print("⚠️ DB_PASSWORD is not set. Please re-run the 'Create the Cloud SQL Instance' cell and set the password.")
else:
    print("✅ Cloud SQL environment variables configured.")

### Define the Session Service Builder for Cloud SQL

This function defines the session service builder that connects to the Cloud SQL for PostgreSQL instance. It constructs a PostgreSQL connection string using the credentials and host details configured previously, enabling the agent to use Cloud SQL for conversation memory.

In [ ]:
def create_database_session_service_sql() -> BaseSessionService:
    db_url = f"postgresql+pg8000://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

    session_db_kwargs = {
        "pool_recycle": 3600,
        "echo": False
    }

    print(f"Creating DatabaseSessionService with Cloud SQL URL.")
    return DatabaseSessionService(db_url=db_url, **session_db_kwargs)

### Create the AdkApp Instance with Cloud SQL Session

This cell creates a new `AdkApp` instance configured to use our Cloud SQL database for session management. By swapping the session service builder, all subsequent conversations with this `sql_app` will be stored in PostgreSQL.

In [ ]:
sql_app = AdkApp(
    agent=root_agent,
    session_service_builder=create_database_session_service_sql,
)

### Create a New Conversation Session (Cloud SQL)

Similar to the SQLite setup, this step creates a new session specifically for interactions that will be backed by the Cloud SQL database.

In [ ]:
user_id = "sql_user_demo"

session = sql_app.create_session(user_id=user_id)

### Interact with the Agent (Using Cloud SQL)

This cell sends a message to the agent using the Cloud SQL-backed application instance. The conversation turn will be saved to your PostgreSQL database in Cloud SQL, demonstrating its use as a persistent memory store.

In [ ]:
print("\n--- User's Message ---")
message = "Hello whats up?"
print(f"You: {message}\n")


print("--- Agent's Response ---")
for event in sql_app.stream_query(user_id=user_id, session_id=session.id, message=message):
    print(event['content'])